# Hybrid Ensemble Method

This method takes all 4 models and performs majority voting on each image. The class with the most votes wins and the model then selects the highest softmax score for that guess from the models. The rest of the classes are an average of all the softmax values.

In [ ]:
import os
import csv
import torch
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import logging
import numpy as np

# Configure Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Dataset Paths- Update with your paths
test_dir = r'C:\Users\blake\OneDrive\Desktop\MCS\Artifical Neural Networks\distracted-driving-behaviors\state-farm-distracted-driver-detection\imgs\test'
submission_file = "hybrid_ensemble_submission.csv"

# Model Checkpoints- update paths with your paths
models_list = [
    ("C:/Users/blake/OneDrive/Desktop/MCS/Artifical Neural Networks/distracted-driving-behaviors/Custom CNN/custom_model_best.pth", "CustomCNN"),
    ("C:/Users/blake/OneDrive/Desktop/MCS/Artifical Neural Networks/distracted-driving-behaviors/Non-Pre-Trained-VGG/custom_vgg_best_model.pth", "CustomVGG"),
    ("C:/Users/blake/OneDrive/Desktop/MCS/Artifical Neural Networks/distracted-driving-behaviors/Resnet/best_resnet_model.pth", "ResNet18"),
    ("C:/Users/blake/OneDrive/Desktop/MCS/Artifical Neural Networks/distracted-driving-behaviors/Pretrained_VGG/best_vgg_model.pth", "VGG16"),
]

# Hyperparameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
NUM_CLASSES = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

# Transformations
transform_test = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization
])

# Custom Models
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.3),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.3),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.3)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * (IMG_HEIGHT // 8) * (IMG_WIDTH // 8), 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Custom VGG-like Model
class CustomVGG(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomVGG, self).__init__()
        
        # Feature Extractor
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def get_resnet_model(num_classes):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def get_vgg_model(num_classes):
    model = models.vgg16(pretrained=False)
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    return model

# Load Models
model_classes = {
    "CustomCNN": CustomCNN(NUM_CLASSES),
    "CustomVGG": CustomVGG(NUM_CLASSES),
    "ResNet18": get_resnet_model(NUM_CLASSES),
    "VGG16": get_vgg_model(NUM_CLASSES),
}
models = []
for model_path, model_name in models_list:
    model = model_classes[model_name]
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    models.append(model)
logging.info("All models loaded successfully.")

# Generate Predictions with Majority Voting
logging.info("Generating predictions...")
fieldnames = ["img"] + [f"c{i}" for i in range(NUM_CLASSES)]

test_image_paths = [os.path.join(test_dir, img) for img in os.listdir(test_dir) if img.endswith(".jpg")]

with open(submission_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    for i in tqdm(range(0, len(test_image_paths), BATCH_SIZE), desc="Processing Batches", unit="batch"):
        batch_paths = test_image_paths[i:i + BATCH_SIZE]
        images = []
        img_names = []

        for img_path in batch_paths:
            img_name = os.path.basename(img_path)
            img = Image.open(img_path).convert("RGB")
            img = transform_test(img)
            images.append(img)
            img_names.append(img_name)

        images = torch.stack(images).to(device)

        with torch.no_grad():
            # Collect predictions from all models
            outputs = [torch.softmax(model(images), dim=1).cpu().numpy() for model in models]
            avg_outputs = np.mean(outputs, axis=0)  # Majority Voting (can be tweaked)

        for j, img_name in enumerate(img_names):
            row = {"img": img_name}
            row.update({f"c{k}": avg_outputs[j][k] for k in range(NUM_CLASSES)})
            writer.writerow(row)

logging.info(f"Submission file '{submission_file}' created successfully.")


2024-12-04 21:12:52,809 - INFO - Using device: cuda
c:\Users\blake\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\blake\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\blake\AppData\Local\Temp\ipykernel_16176\3121278071.py:159: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURI